# Preprocess
Preprocess images.

## Load libraries

In [1]:
# Load libraries
import os, sys
from pathlib import Path

import cv2
import numpy as np
import pandas as pd

# Relative imports
module_path = os.path.abspath(os.path.join('..', '..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from Normalizers.MixedNormalizer import MixedNormalizer

## Configuration

### Calibration parameters
**!Note:** Parameters were obtained using camera calibration using OpenCv library. For more information about camera calibration see:
- https://www.pythonpool.com/opencv-solvepnp/
- https://learnopencv.com/head-pose-estimation-using-opencv-and-dlib/

In [3]:
camera_matrix = np.array([[1.49454593e+03, 0., 9.55794289e+02],
                         [0., 1.49048883e+03, 5.18040731e+02],
                         [0., 0., 1.]], dtype = 'float64')

camera_distortion = np.array([[ 0.08599595, -0.37972518, -0.0059906,  -0.00468435,  0.45227431]], dtype = 'float64')

### Directories

In [4]:
data_directory = '../../Datasets/Study_2/Meta.tsv'
base_path = '../../Datasets/Study_2/Participants'

output_data_directory = '../../Datasets/Study_2/Meta.tsv'
normalised_directory = 'Normalised'

## Normalisation

### Load meta

In [ ]:
df = pd.read_csv(data_directory, header=0, delimiter='\t')

### Execute normalisation

In [34]:
normalizer = MixedNormalizer(base_path='../../')
normalizer.set_calibration_parameters(camera_matrix, camera_distortion)

total_error_images = 0

for index, row in df.iterrows():
    if index % 200 == 0:
        print(f'Currently processing row: {index}')
    image_path = f'{base_path}/{row["Participant"]}/{row["Case"]}/{row["File"]}'
    try:
        processed_eyes, _, _ = normalizer.normalize_image(image_path)
        
        # Create directories if not exists
        Path(f'{base_path}/{row["Participant"]}/{row["Case"]}/{normalised_directory}').mkdir(parents=True, exist_ok=True)
        
        # Save images
        cv2.imwrite(f'{base_path}/{row["Participant"]}/{row["Case"]}/{normalised_directory}/{index}_right_eye.jpg', processed_eyes[0])
        cv2.imwrite(f'{base_path}/{row["Participant"]}/{row["Case"]}/{normalised_directory}/{index}_left_eye.jpg', processed_eyes[1])
        
        # Update dataframe
        df.loc[index, 'Right_eye_normalized'] = f'{index}_right_eye.jpg'
        df.loc[index, 'Left_eye_normalized'] = f'{index}_left_eye.jpg'
        
    except Exception as e:
        total_error_images += 1
        print(f'{e}, Metadata: (P: {row["Participant"]}, C: {row["Case"]}, F: {row["File"]})')
        

Currently processing row: 0
Currently processing row: 200
Currently processing row: 400
Currently processing row: 600
Currently processing row: 800
Currently processing row: 1000
Currently processing row: 1200
Currently processing row: 1400
Currently processing row: 1600
Currently processing row: 1800
Currently processing row: 2000
Currently processing row: 2200
Currently processing row: 2400
Currently processing row: 2600
Currently processing row: 2800
Currently processing row: 3000
Currently processing row: 3200
Currently processing row: 3400
Currently processing row: 3600
Currently processing row: 3800
Currently processing row: 4000
Currently processing row: 4200
Currently processing row: 4400
Currently processing row: 4600
Currently processing row: 4800


### Save updated dataframe
New dataframe contains normalised file names.

In [10]:
# Save dataframe
df.to_csv(output_data_directory, sep='\t', index=False)